# S&P 2019 Data Wrangling

**by Marc Angelo Acebedo**

## Table of Contents
- [Introduction](#introduction)
- [Data Wrangling](#wrangling)
    - [Gather](#gather)
    - [Assess](#assess)
    - [Clean](#clean)
    - [Store](#store)

## I) Introduction <a id = "introduction">

**Broad question:** How do price forecasts for each firm in the S&P 2019 Index compare to their corresponding actual prices?

**Approach:** Analyze difference in means between average forecast EPS and average actual EPS for each firm.

I will be analyzing quarterly price returns within the past 20 years for the firms present in the S&P 500 2019 Index.

> At first, I wanted to analyze the forecasted vs. actual price earnings of the S&P in its entirety for the past 20 years. However, considering that firms continuously enter and leave stock indices every year, there would be varying levels of inconsistencies and marginal errors when comparing annual S&P returns alone. To combat this problem, I have isolated these two approaches:
- Analyze the historical earnings of *only* the firms present in the S&P 2019 Index
- Keep track of all firms that were present in the S&P for the past 20 years. Keep track of how many times each firm appeared in the Index and for those with the least count, analyze them individually on how they differ from the firms that stayed for longer.

[TK] HEre is a breakdown of my final clean CSV's features, TK.csv.

## II) Data Wrangling <a id="wrangling"></a>

To gather the data depicted under the `./data` folder, I used Bloomberg Excel functions.

### A) Gather <a id = "gather"></a>
> **APPROACH 1:** Focus on the firms that appear in the 2019 S&P Index and analyze their forecasted vs. actual price earnings for the last 20 years.

To ensure consistency in analysis among multiple firms, I divide both the forecasted and actual price earning dates by *calendar period* instead of fiscal period. This is because fiscal period differs by firm whereas calendar period is consistent by dates. 

#### Through the Bloomberg Excel functions, I gathered four datasets with different purposes:

- historical forecasted EPS
- historical actual EPS
- historical actual EOD price
- historical forecasted EPS relying on terms

---
Before delving into the data, let's define what above terms:

**EPS**

> EPS stands for ***Estimated Price Earnings.*** The formal definition of EPS given by Investopedia is this:

Earnings per share is the portion of a company's profit that is allocated to each outstanding share of a common stock, serving as an indicator of the company's financial health.

In other words, the EPS is a portion of the company's **net income** after all of their dividends are paid off. Dividends are profits that are paid out to shareholders of the company. EPS is one of the most useful and valuable financial measurements because they ***determine a stock's worth.*** The higher the stock, the more the company can pay out dividends to its shareholders, and the more net profit they are determined to generate.

$$ EPS = \frac{Net Income - Preferred Dividends}{Weighted Average Common Shares Outstanding}\$$

**EOD**

> EOD stands for the ***End of Day*** price. For any given day, the EOD marks the ***price at which the stock was valued*** at the end of the day's trading period.

In [3]:
PATH = './data/'
PATH_CLEAN = './data/clean/'
PATH_CLEAN_AVGS = './data/clean/averages/'
PATH_CLEAN_AVGS_SUB = './data/clean/averages/components/'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from functools import reduce

### Import Data <a id = "import"></a>

### Let's summarize the contents of the following DataFrames as we import them.
> All DataFrames consist of the 505 firms found in the 2019 S&P Index with EPS and EOD data encompassing 20 years: from January 1999 until the December 2019.

**Historic forecasted EPS**
> According to Investopedia, consensus estimates is normally an average or median of all the forecasts from individual analysts tracking a particular stock. In this case, the consensus estimate is for ***EPS for each firm present in the index as of 2019.*** Forecasted EPS is calculated by ***quarterly earnings,*** usually by each firm's fiscal period. Estimates of quarterly earnings are published at the beginning of each quarterly period.

In [4]:
#historic forecasted EPS 
eps_fc = pd.read_csv(PATH + 'sp-eps-fc.csv')

In [5]:
eps_fc.head()

,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,1999Q1,NaN,NaN,NaN,0.025,NaN,NaN,-0.10,0.423,NaN,...,0.456,0.090,0.239,0.123,NaN,NaN,0.123,NaN,0.622,NaN
1,1999Q2,NaN,NaN,NaN,0.020,NaN,NaN,-0.13,0.420,NaN,...,0.213,0.090,0.264,0.135,NaN,NaN,0.150,NaN,0.661,NaN
2,1999Q3,NaN,NaN,NaN,0.023,NaN,NaN,-0.11,0.381,NaN,...,0.767,0.099,0.300,0.129,NaN,NaN,0.171,NaN,0.693,NaN
3,1999Q4,NaN,NaN,NaN,0.016,NaN,NaN,NaN,0.432,NaN,...,0.432,0.124,0.386,0.178,NaN,NaN,0.164,NaN,0.667,NaN
4,00Q1,NaN,NaN,NaN,0.032,NaN,NaN,NaN,0.441,NaN,...,0.283,0.143,0.436,0.138,NaN,NaN,0.138,NaN,0.668,NaN


**Historic actual EPS**
> Unlike forecasted EPS, actual EPS are the real numbers denoting Earnings-per-Share for a singular firm. Historic actual EPS will be compared to forecasted EPS to draw correlations and comparisons.

In [6]:
#historic actual EPS
eps_act = pd.read_csv(PATH + 'sp-eps-act.csv')

In [7]:
eps_act.head()

,Quarter,Year,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,Q1,1999,0.16,0.99,NaN,0.040000,NaN,0.0925,-0.030,0.44,...,0.34,0.0000,0.210,0.123333,1.64,NaN,0.1725,NaN,0.56,NaN
1,Q2,1999,0.35,1.76,NaN,0.035357,NaN,0.1000,-0.145,0.42,...,0.06,0.0975,0.285,0.133333,2.28,NaN,0.2900,NaN,0.60,NaN
2,Q3,1999,0.30,1.86,NaN,0.050357,NaN,0.1075,-0.135,0.30,...,0.63,0.0325,0.315,0.130000,1.96,NaN,0.3200,NaN,0.64,NaN
3,Q4,1999,0.32,1.89,NaN,0.024643,NaN,0.0375,-0.080,0.43,...,0.43,0.1300,0.600,0.180000,1.72,NaN,0.2375,NaN,0.49,NaN
4,Q1,2000,0.29,0.89,NaN,0.040585,NaN,0.1050,-0.045,0.45,...,0.45,0.1650,0.500,0.140000,-1.48,NaN,0.2025,NaN,-0.33,NaN


**Historic actual EOD**
> Though this is not directly related to EPS data, EOD would be an interesting measure to use when generating intriguing visualizations and analyses. Who knows what visuals and conclusions I would arrive to with this measure. 

In [8]:
#historic actual EOD
eod_act = pd.read_csv(PATH + 'sp-eod-act.csv')

In [9]:
eod_act.head()

,date,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,3/31/1999,NaN,NaN,NaN,1.2835,NaN,8.2934,6.250,20.9504,NaN,...,NaN,20.2813,35.2813,7.7500,140.6214,NaN,12.6284,NaN,66.5000,NaN
1,6/30/1999,NaN,NaN,NaN,1.6540,NaN,6.1859,6.875,20.3630,NaN,...,NaN,28.6250,38.5625,9.6250,155.6057,NaN,9.7297,NaN,63.5000,NaN
2,9/30/1999,NaN,NaN,NaN,2.2612,NaN,5.7462,7.750,16.4471,NaN,...,NaN,32.7656,37.9688,7.5833,110.4883,NaN,7.3591,NaN,55.1250,NaN
3,12/31/1999,52.0909,NaN,NaN,3.6719,NaN,3.6843,18.375,16.2513,NaN,...,NaN,45.4688,40.2813,7.8750,59.7723,NaN,6.9434,NaN,59.1875,NaN
4,3/31/2000,70.0721,NaN,NaN,4.8504,NaN,3.6388,20.250,15.7478,NaN,...,NaN,82.8125,38.9063,9.4583,68.4994,NaN,5.5839,NaN,41.6250,NaN


**Historic forecasted EPS 3 months prior**
> Instead of using forecast data collected at the beginning of the fiscal period, this feature contains EPS data projected 3 months before the current fiscal period. This is an interesting metric to see how differently forecasters make their predictions at different times.

In [10]:
#historic forecasted EPS 3-months prior
eps_fc_terms = pd.read_csv(PATH + 'sp-eps-fc-terms.csv')

In [11]:
eps_fc_terms.head()

,Forecast Made,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,10/1/1999,00Q1,NaN,NaN,NaN,0.030,NaN,NaN,NaN,0.466,...,NaN,0.143,0.398,0.140,NaN,NaN,0.144,NaN,0.777,NaN
1,1/1/2000,00Q2,NaN,NaN,NaN,0.026,NaN,NaN,NaN,0.437,...,NaN,0.168,0.378,0.153,NaN,NaN,0.184,NaN,0.790,NaN
2,4/1/2000,00Q3,NaN,NaN,NaN,0.029,NaN,NaN,NaN,0.414,...,0.84,0.189,0.425,0.149,NaN,NaN,0.218,NaN,0.803,NaN
3,7/1/2000,00Q4,NaN,NaN,NaN,0.032,NaN,NaN,-0.18,0.477,...,0.53,0.211,0.487,0.200,NaN,NaN,0.244,NaN,0.777,NaN
4,9/1/2000,01Q1,NaN,NaN,NaN,0.041,NaN,NaN,-0.09,0.495,...,0.40,0.259,0.475,0.158,NaN,NaN,0.178,NaN,0.750,NaN


### B) Assess <a id = "assess"></a>

> The following DataFrames contain data for each firm across various dates. To account for all firm averages, my goal is to generate a CSV file where each row contains the firm average, with the features as columns.

In [12]:
dict_dfs = {'eps_fcast' : eps_fc,
           'eps_actual' : eps_act,
           'eod_actual' : eod_act,
           'eps_fcast_terms' : eps_fc_terms}

### Check Missing Data

In [13]:
for key, df in dict_dfs.items():
    print(key, df.shape)

eps_fcast (84, 506)
eps_actual (84, 507)
eod_actual (84, 506)
eps_fcast_terms (80, 507)


***We need to make sure the number of firms in each DataFrame is consistent.***

---

**Observation 1:** for `eps_fcast`, there are 505 firms encompassing 84 quarterly fiscal periods since 1999.

> There are 506 columns: 1 column being `Term Forecast`, the rest firm names.

**Observation 2:** for `eps_actual`, there are 505 firms encompassing 84 quarterly fiscal periods since 1999.

> There are 507 columns: 2 columns being `Quarter` and `Year`, the rest firm names.

**Observation 3:** for `eod_actual`, there are 505 firms encompassing 84 quarterly calendar periods since 1999.

> There are 506 columns: 1 column being `date`, the rest firm names.

**Observation 4:** for `eps_fcast_terms`, there are 505 firms encompassing 80 quarterly calendar periods since 1999.

> There are 507 columns: 2 columns being `Forecast Made` and `Term Forecasted`, the rest firm names.
 
Since there are only 80 quarterly calendar periods, that ***implies an entire year is missing.***

**Observation 5:** For `eps_fcast`, `eps_actual`, and `eod_actual`, since there are 4 quarters in a year, 84 quarterly forecast periods equate to 21 years. This is correct since we are analyzing the years from 1999 until the end of 2019.

### Most importantly, the number of firms across all DataFrames is consistent.

----

***Check which year is missing from `eps_fcast_terms`***.

> Each **Term Forecasted** entry under `eps_fcast_terms` records the year with 2 digits, so Quarter 1 of the year 2000 becomes 00Q1.
- isolate the first 2 characters to get the year
- join a '20' in front of the string so 00 becomes 2000
- list the number of unique values.

In [14]:
#iterate over all years in eps_fcast_terms, append '20' in front of string
list(map(lambda x: '20' + x, eps_fc_terms['Term Forecasted'].str[:2].unique()))

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']

**Observation**: The year 1999 is missing from `eps_fc_terms`. This makes sense because the start of the forecasting period would be in the last quarter of 1999, which is October. 

---

### Check Nulls

In [15]:
for key, df in dict_dfs.items():
    #display (row, column) per DataFrame
    print('# NaN in {}: {}'.format(key, df.isna().sum().sum()))

# NaN in eps_fcast: 7055
# NaN in eps_actual: 5021
# NaN in eod_actual: 6921
# NaN in eps_fcast_terms: 7080


**Observation:** All four DataSets contain null values.
> In order to combat this, we'll have to look at both the **number of rows** and **number of columns** with missing data, separately. This way, we can isolate which firms and/or time periods contain complete or incomplete data.

In [16]:
#check rows for missing data
for key, df in dict_dfs.items():
    num_rows_missing = df.isna().sum().max()
    print('{} has {} time periods containing missing data, out of {} total rows.'.format(key, num_rows_missing, df.shape[0]))

eps_fcast has 82 time periods containing missing data, out of 84 total rows.
eps_actual has 84 time periods containing missing data, out of 84 total rows.
eod_actual has 83 time periods containing missing data, out of 84 total rows.
eps_fcast_terms has 80 time periods containing missing data, out of 80 total rows.


In [17]:
len(eps_fc.columns[eps_fc.isnull().any()])

254

In [18]:
#check columns for missing data
for key, df in dict_dfs.items():
    cols_missing = df.columns[df.isnull().any()]
    num_cols_missing = len(cols_missing)
    print('{} has {} firms containing missing data, out of {} total columns.'
         .format(key, num_cols_missing, df.shape[1]))

eps_fcast has 254 firms containing missing data, out of 506 total columns.
eps_actual has 432 firms containing missing data, out of 507 total columns.
eod_actual has 505 firms containing missing data, out of 506 total columns.
eps_fcast_terms has 340 firms containing missing data, out of 507 total columns.


**Observation 1:** The only two datasets with incomplete data for all time periods are **actual EPS** and **forecasted EPS 3 months prior.**

> To address this problem, it'd be helpful to isolate the time period ranges for the datasets with incomplete data by row, **forecasted EPS** and **actual EOD price.**

**Observation 2:** For all datasets, all firms contain incomplete data across all time periods.
> This is expected, as analyzing financial history spanning over 20 years will naturally be rife with missing and inaccurate data. The ***good news is that `eps_fcast`, `eps_actual`, and `eps_fcast_terms` are the most complete, while `eod_actual` contains the most amount of missing data.***


**Moving forward, we need to make sure that these inconsistencies won't clash with our analysis.**
> **My approach:** instead of looking at rows and columns ***with*** missing data, we'll be looking at rows and columns that ***are all missing data.***

I figured that if there is some missing data here and there scattered throughout the matrix, then that should not skew our analysis too much.

However, if there a significant amount of rows/columns that are entirely empty, then we ***might have to get ready to drop some dates and firms from our data overall.***

In [19]:
#check for empty rows, return False if row contains at least one non-null value, True if all are null
for key, df in dict_dfs.items():
    cols_check = df.columns
    num_empty_rows = (df[cols_check].isnull().apply(lambda x: all(x), axis = 1)).value_counts()
    print(key, '\n', num_empty_rows, '\n----')

eps_fcast 
 False    84
dtype: int64 
----
eps_actual 
 False    84
dtype: int64 
----
eod_actual 
 False    84
dtype: int64 
----
eps_fcast_terms 
 False    80
dtype: int64 
----


**Observation:** All datasets do not contain empty rows.
> This is good news, since we can rely on the firms' averages per row instead of having to drop or limit time periods.

In [20]:
#check for empty columns, return False if column contains at least one non-null value, True if all are null
for key, df in dict_dfs.items():
    cols_check = df.columns
    num_empty_cols = df[cols_check].isnull().apply(lambda x: all(x), axis = 0).value_counts()
    print(key, '\n', num_empty_cols)

eps_fcast 
 False    506
dtype: int64
eps_actual 
 False    506
True       1
dtype: int64
eod_actual 
 False    506
dtype: int64
eps_fcast_terms 
 False    506
True       1
dtype: int64


**Observation:** `eps_actual` and `eps_fcast_terms` are the only datasets that have an empty column.
> Let's isolate and look at the singular empty column for both DataFrames.

In [21]:
#create function to return an array of column names containing empty data
def comb_cols(df):
    empty_cols = []
    for column in df:
        if df[column].isnull().all():
            empty_cols.append(column)
            
    return empty_cols

In [22]:
#comb datasets for empty columns
print('In eps_act, the firm {} has no data.'.format(comb_cols(eps_act)))
print('In eps_fc_terms, the firm {} has no data.'.format(comb_cols(eps_fc_terms)))

In eps_act, the firm ['AMCR UN Equity'] has no data.
In eps_fc_terms, the firm ['AMCR UN Equity'] has no data.


**Observation:** The same firm in both datasets is empty of data.
> Though this is an annoying error to deal with, it still is to our advantage that both datasets ***share one firm*** in common for missing data. This way, we don't have to worry about dropping two entire firms.

---

### Check Duplicate Data

In [23]:
#check for duplicate data across all rows and columns
for key, df in dict_dfs.items():
    print(key, df.duplicated().sum())

eps_fcast 0
eps_actual 0
eod_actual 0
eps_fcast_terms 0


**Observation:** For all datasets, there are ***no duplicate data.*** This is good news!

**Next, I will check for duplicated firm names.** Although the presence of duplicated firm names will inherently imply duplicated data, sometimes data gets dispersed in weird, unexpected ways, especially when dealing with large datasets.

In [24]:
#check for duplicated firm names
for key, df in dict_dfs.items():
    print(key, df.columns.duplicated().sum())

eps_fcast 0
eps_actual 0
eod_actual 0
eps_fcast_terms 0


**Observation:** For all datasets, there are ***no duplicate firm names.*** This is also good news.

### There is no need to dedupe our data during the cleaning stage.

---

### Check Data Types

> Make sure all numerical data types are consistent.
- EPS and EOD values must all be **floats**
- Dates should all be **objects** (for now)

- `eps_fc` contains ***1 date field,***, so we should expect ***1 object type.***
- `eps_act` contains ***2 date fields,*** so we should expect ***2 object types.***
- `eod_act` contains ***1 date field,*** so we should expect ***1 object type.***
- `eps_fc_terms` contains ***2 date fields,*** so we should expect ***2 object types.***

> And everything else should be a ***float*** type.

**Examine each dataset's overall data types per column.**

In [25]:
for key, df in dict_dfs.items():
    print(key,'\n', df.dtypes.value_counts(), '\n--------')

eps_fcast 
 float64    505
object       1
dtype: int64 
--------
eps_actual 
 float64    499
object       7
int64        1
dtype: int64 
--------
eod_actual 
 float64    505
object       1
dtype: int64 
--------
eps_fcast_terms 
 float64    505
object       2
dtype: int64 
--------


**Observation 1:** `eps_fc`, `eod_act`, and `eps_fc_terms` are all consistent with the expected number of object types.

> But we'll still have to double check these facts later.

**Observation 2:** `eps_act` contains 5 more object columns than expected.

---

***Isolate 'Object' Columns under `eps_act`***

In [26]:
eps_act.select_dtypes(include = 'object').head()

,Quarter,BRK/B UN Equity,FOX UW Equity,GOOG UW Equity,HCP UN Equity,SYMC UW Equity,UA UN Equity
0,Q1,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
1,Q2,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
2,Q3,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
3,Q4,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
4,Q1,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable


**Observation:** These firms are all 'Object' types because there is no recorded data under them.
> This is actually missing data. From `eps_act`, there are 6 firms that are empty of data.

### This was a tricky situation to spot, but I caught it in the end. There actually is missing data after all, but we missed those 5 entire firms during the "Check Missing Data" stage, because they were recorded as objects. This means that Pandas wrongly recognized these 5 firms as "complete."

### Quality

**Missing Data**
> **In the end, I decided not to drop any rows or columns with missing data. For the sake of data preservation, I kept all missing data recorded as NaN.**

-  `eps_fc_terms` is missing the year 1999.
- `eps_act` and `eps_fc_act` have one firm with empty data: 'AMCR UN Equity'
- `eps_act` contains 7 empty firms: BRK/B UN Equity, FOX UW Equity, GOOG UW Equity, HCP UN Equity, SYMC UW Equity, UA UN Equity
--- 
- Unnormalized date formats among all DataFrames.
- Firm names across all DataFrames are capitalized and contain white space.
- Erroneous data type for `eps_act` Object columns.
- Erroneous data type for **date** under `eod_act`
- Erroneous data type for **forecast_made** under `eps_fc_terms`
- Erroneous data types for **term** under `eps_act`, `eps_fc`, and `eps_fc_terms` to DateTime quarter index
- No recorded quarterly data for `eod_act`
---
- Firm names not referenced by **firm_id**
- No recorded 20-year for each dataset.
- No recorded yearly averages for each dataset
- No recorded quarterly averages for each dataset.
---
- Features datasets 

### Tidiness
- Unnormalized data among `df_twenty_year_avgs`, `df_yearly_avgs`, and `df_quarter_avgs`
- 20-year, yearly, and quarterly averages contained in different DataFrames.
- Unnormalized data among `eps_act`, `eps_fc`, `eod_act`, and `eps_fc_terms`


### C) CLEAN <a id="clean"></a>

In [407]:
eps_fc_clean = eps_fc.copy()
eps_act_clean = eps_act.copy()
eod_act_clean = eod_act.copy()
eps_fc_terms_clean = eps_fc_terms.copy()

In [408]:
#create dictionary of all clean dfs for iteration
dict_clean = {'eps_fc_clean' : eps_fc_clean,
             'eps_act_clean' : eps_act_clean,
             'eod_act_clean' : eod_act_clean,
             'eps_fc_terms_clean' : eps_fc_terms_clean}

### Code
> **ISSUE 1:**  Unnormalized date formats among all DataFrames.

**Define:** 
- Conjoin `eps_act_clean` dates from 2 columns into 1 to match `eps_fc_clean` format.
> The format we want is YYYYQN. For example, Quarter 1 in 2005 will be 2005Q1.

- Rename both fiscal periods under `eps_act_clean` and `eps_fc_clean` to **term.**

- For both fiscal periods under `eps_fc_terms_clean`, add an underscore.
- Under `eps_fc_terms_clean`, format **term** to YYYYQN.

In [409]:
#create new column, combine year and quarter into 1 string
eps_act_clean['term'] = eps_act_clean['Year'].map(str) + eps_act_clean['Quarter']

In [410]:
#drop 'Year' and 'Quarter'
eps_act_clean.drop(['Year', 'Quarter'], axis = 1, inplace = True)

In [411]:
#designate new column as first column
reorder_cols = eps_act_clean.columns.tolist()
reorder_cols.insert(0, reorder_cols.pop(reorder_cols.index('term')))

eps_act_clean = eps_act_clean.reindex(columns = reorder_cols)

In [412]:
#rename eps_fc_clean fiscal period to term
eps_fc_clean.rename(index = str, columns = {'Term Forecasted' : 'term'}, inplace = True)

#reassign to dictionary
dict_clean['eps_act_clean'] = eps_act_clean

In [413]:
eps_fc_clean[4:].term = '20' + eps_fc_clean[4:].term.astype(str)

D:\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [414]:
#add an underscore for both date periods under eps_fc_terms_clean
eps_fc_terms_clean.rename(index = str, columns = {'Forecast Made' : 'forecast_made',
                                                 'Term Forecasted' : 'term'},
                         inplace = True)

In [415]:
#under term, add a '20' before each string
eps_fc_terms_clean.term = '20' + eps_fc_terms_clean.term.astype(str)

### Test

In [416]:
#give overview of both DataFrames
eps_fc_clean.sample(3)

,term,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
48,2011Q1,0.578,-1.325,1.369,0.770,NaN,0.534,-0.140,0.903,0.756,...,0.421,0.533,2.066,0.477,NaN,NaN,0.642,1.122,NaN,NaN
11,2001Q4,NaN,NaN,0.010,0.012,NaN,NaN,-0.175,0.520,0.105,...,0.438,0.226,0.396,0.191,NaN,NaN,0.260,0.265,0.829,NaN
74,2017Q3,0.522,1.397,1.207,1.572,1.385,1.374,0.290,0.653,1.522,...,0.922,0.490,0.870,0.660,0.826,0.631,0.668,1.740,0.719,0.622


In [417]:
eps_act_clean.sample(3)

,term,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
82,2019Q3,0.61,0.96,1.76,2.20,1.27,1.44,1.00,0.54,1.96,...,1.02,0.95,0.75,0.38,0.99,0.36,0.83,2.10,1.23,0.91
81,2019Q2,0.57,1.49,1.74,2.47,0.49,0.13,1.11,0.57,1.76,...,0.46,0.85,0.73,0.16,0.79,0.77,0.94,0.65,1.05,0.77
68,2016Q1,0.37,1.15,2.16,3.30,0.83,1.60,0.21,0.21,1.31,...,0.47,0.57,0.43,0.72,0.12,0.37,0.88,0.54,0.38,0.41


In [418]:
#check consistent format between both `term` fields
forecasts = pd.concat([eps_fc_clean.term, eps_act_clean.term])

#count number of instances where unique term values are the same
(forecasts.value_counts().sort_index() == 2).value_counts()

True    84
Name: term, dtype: int64

In [419]:
#check that dates are the first columns
for key, df in dict_clean.items():
    print(df.columns[:2])

Index(['term', 'A UN Equity'], dtype='object')
Index(['term', 'A UN Equity'], dtype='object')
Index(['date', 'A UN Equity'], dtype='object')
Index(['forecast_made', 'term'], dtype='object')


## The **term** column under `eps_fc_clean`, `eps_act_clean`, and `eps_fc_terms_clean` are all normalized to YYYQN format. This means `eod_act_clean` is the only one with a different date formatting rule.

### Code
> **ISSUE 2:** Firm names across all DataFrames are capitalized and contain white space.

**Define:** Iterate across all DataFrames. Get rid of everything from the first whitespace character onward. Lowercase column names.

In [420]:
#lowercase names
for key, df in dict_clean.items():
    lower_cols = [x.lower() for x in df.columns]
    df.columns = lower_cols

In [421]:
#get rid of white space
for key, df in dict_clean.items():
    stock_symbols = [x.split(' ', 1)[0] for x in df.columns]
    df.columns = stock_symbols

### Test

In [422]:
#check that all firm names are consistent, should return 505 firm results
all_ticks = []
for key, df in dict_clean.items():
    firm_ticks = np.array(df.columns.values)
    all_ticks.append(firm_ticks)
    
#flatten array
all_ticks = np.hstack(all_ticks)

#assign tick names to Pandas Series
all_ticks = pd.Series(all_ticks)

In [423]:
#look at lowest 3 value counts, SHOULD BE DATE PERIODS
all_ticks.value_counts().sort_values()[:5]

forecast_made    1
date             1
term             3
acn              4
wat              4
dtype: int64

In [424]:
#look at highest value counts, should be FOUR (because there are 4 dataframes all with firm names)
all_ticks.value_counts().sort_values(ascending = False)[:5]

khc     4
wmt     4
pcar    4
mlm     4
lkq     4
dtype: int64

In [425]:
#check for any upper case instance among columns
all_ticks.str.isupper().any()

False

In [426]:
#check for any white space
all_ticks.str.contains(' ').any()

False

### Code
> **ISSUE 3:** Erroneous data type for `eps_act` Object columns.

**Define:**

- Convert '#N/A Field Not Applicable' strings into 'NaN'
- Convert the 6 `eps_act_clean` Object columns into NaN type.

In [427]:
#isolate 6 firm names recorded as 'object'
obj_ticks = eps_act_clean.select_dtypes(include = 'object').columns.values

#exclude 'term'
obj_ticks = obj_ticks[1:]

In [428]:
#convert to NaN
str_replace = '#N/A Field Not Applicable'
eps_act_clean[obj_ticks] = eps_act_clean[obj_ticks].replace(str_replace, np.nan)

### Test

In [429]:
eps_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 506 entries, term to zts
dtypes: float64(505), object(1)
memory usage: 332.1+ KB


In [430]:
#make sure 'term' is the only object-type column
eps_act_clean.select_dtypes(include = 'object').columns.values.tolist()

['term']

### Code

> **ISSUE 4:**  Erroneous data type for **date** under `eod_act`

**Define:** Convert column `date` to DateTime object.

In [431]:
eod_act_clean.date = pd.to_datetime(eod_act_clean.date, infer_datetime_format=True)

### Test

In [432]:
eod_act_clean.sample(2)

,date,a,aal,aap,aapl,abbv,abc,abmd,abt,acn,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
12,2002-03-28,23.5550,NaN,15.333,1.6907,NaN,16.5685,11.10,23.5406,26.70,...,NaN,39.86,43.83,18.53,28.3219,NaN,10.5665,34.05,59.27,NaN
60,2014-03-31,39.9877,36.6,126.500,76.6771,51.4,65.5900,26.04,38.5100,79.72,...,NaN,54.27,97.68,46.04,29.7709,36.42,54.2095,94.58,30.98,28.94


In [433]:
#confirm datetime64 present
eod_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 506 entries, date to zts
dtypes: datetime64[ns](1), float64(505)
memory usage: 332.1 KB


### Code
> **ISSUE 5:** Erroneous data type for **forecast_made** under `eps_fc_terms`.

**Define:**

- Convert column **forecast_made** to DateTime object.

In [434]:
eps_fc_terms_clean.forecast_made = pd.to_datetime(eps_fc_terms_clean.forecast_made, infer_datetime_format=True)

### Test

In [435]:
#confirm presence of datetime64 object
eps_fc_terms_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 0 to 79
Columns: 507 entries, forecast_made to zts
dtypes: datetime64[ns](1), float64(505), object(1)
memory usage: 317.5+ KB


In [436]:
eps_fc_terms_clean.sample(2)

,forecast_made,term,a,aal,aap,aapl,abbv,abc,abmd,abt,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
5,2001-01-01,2001Q2,NaN,NaN,NaN,0.006,NaN,NaN,-0.120,0.470,...,NaN,0.311,0.555,0.177,NaN,NaN,0.200,NaN,0.798,NaN
75,2018-07-01,2018Q4,0.729,1.158,1.125,2.657,1.956,1.529,0.642,0.816,...,0.404,0.830,1.309,0.829,1.095,0.895,0.966,2.135,0.997,0.832


### Code
> **ISSUE 6:** Erroneous data types for **term** under `eps_act`, `eps_fc`, and `eps_fc_terms` to DateTime quarter index

**Define:** Convert YYYYQQ formats into DateTime quarter index type

In [437]:
#helper function to turn a column into Quarter Period type
def convert_period(df, col):
    return pd.to_datetime(df[col]).dt.to_period('Q')
# eps_fc_terms_clean.term = pd.PeriodIndex(eps_fc_terms_clean.term, freq='Q')

In [438]:
#convert term fields to Period type
eps_fc_clean.term = convert_period(eps_fc_clean, 'term')
eps_act_clean.term = convert_period(eps_act_clean, 'term')
eps_fc_terms_clean.term = convert_period(eps_fc_terms_clean, 'term')

### Test

In [439]:
#verify Period Object types
print(eps_fc_clean.dtypes['term'])
print(eps_act_clean.dtypes['term'])
print(eps_fc_terms_clean.dtypes['term'])

period[Q-DEC]
period[Q-DEC]
period[Q-DEC]


### Code
> **ISSUE 7:** No recorded quarterly data for `eod_act`

**Define:** Add a new column **term**

In [440]:
eod_act_clean['term'] = eod_act_clean.date.dt.to_period('Q')

In [441]:
#reorder columns
eod_cols = ['date', 'term'] + [col for col in eod_act_clean.iloc[:, 1:] if col!='term']

In [442]:
eod_act_clean = eod_act_clean.reindex(columns = eod_cols)

### Test

In [443]:
#verify column order
eod_act_clean.head()

,date,term,a,aal,aap,aapl,abbv,abc,abmd,abt,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
0,1999-03-31,1999Q1,NaN,NaN,NaN,1.2835,NaN,8.2934,6.250,20.9504,...,NaN,20.2813,35.2813,7.7500,140.6214,NaN,12.6284,NaN,66.5000,NaN
1,1999-06-30,1999Q2,NaN,NaN,NaN,1.6540,NaN,6.1859,6.875,20.3630,...,NaN,28.6250,38.5625,9.6250,155.6057,NaN,9.7297,NaN,63.5000,NaN
2,1999-09-30,1999Q3,NaN,NaN,NaN,2.2612,NaN,5.7462,7.750,16.4471,...,NaN,32.7656,37.9688,7.5833,110.4883,NaN,7.3591,NaN,55.1250,NaN
3,1999-12-31,1999Q4,52.0909,NaN,NaN,3.6719,NaN,3.6843,18.375,16.2513,...,NaN,45.4688,40.2813,7.8750,59.7723,NaN,6.9434,NaN,59.1875,NaN
4,2000-03-31,2000Q1,70.0721,NaN,NaN,4.8504,NaN,3.6388,20.250,15.7478,...,NaN,82.8125,38.9063,9.4583,68.4994,NaN,5.5839,NaN,41.6250,NaN


In [444]:
#verify 84 columns
eod_act_clean.shape[0]

84

In [445]:
eod_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 507 entries, date to zts
dtypes: datetime64[ns](1), float64(505), period[Q-DEC](1)
memory usage: 332.8 KB


## All full years are normalized to YYYY-MM-DD, and all quarterly records to YYYY-QQ.

## This is to enable for more efficient handling, cleaning, and classifying of data later on.

### Code

**ISSUE 8:** Firm names not referenced by **firm_id**

**Define:**
- Assign a **firm_id** to each firm for future normalization.

- Generate new CSV named `firms.csv`

In [446]:
#get firm names
firm_names = eps_act_clean.columns[1:].to_numpy()

In [447]:
#assign ids to firm names
firm_names_ids = {}
counter = 0

for firm in firm_names:
    firm_names_ids.update({firm: counter})
    counter += 1

In [448]:
#generate new DF
firm_ids = pd.DataFrame(list(firm_names_ids.items())).rename(columns = {0: 'firm', 1: 'firm_id'})

In [449]:
#switch firm and firm_id columns
firm_ids = firm_ids.reindex(columns = ['firm_id', 'firm'])

### Test

In [450]:
firm_ids.sample(10)

,firm_id,firm
460,460,unp
405,405,rsg
363,363,pbct
210,210,gpc
448,448,ttwo
83,83,cb
156,156,ebay
269,269,key
116,116,coty
171,171,etr


In [451]:
firm_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 2 columns):
firm_id    505 non-null int64
firm       505 non-null object
dtypes: int64(1), object(1)
memory usage: 8.0+ KB


### Code
> **ISSUE 9:** No recorded 20-year averages for each dataset.

**Define:**


- Isolate 20-year averages for each firm into its own DataFrame

- Create new DataFrame `twenty_avgs` depicting all 20-year averages for each firm

In [452]:
#convert foreign keys to str temporarily
firm_ids_str = {str(k):str(v) for k, v in firm_names_ids.items()}
# firm_ids_str = {y:x for x, y in firm_ids_str.items()}

In [453]:
#helper function to get 20-year averages
def get_twenty_yr_avg(df, index_names, col_name, firm_ids_str):
    df = pd.DataFrame({col_name : df[index_names].mean()}).rename_axis('firm_id').reset_index()
    df['firm_id'] = df['firm_id'].map(firm_ids_str)
    return df

In [454]:
#get 20-year averages
eps_fc_twenty_avg = get_twenty_yr_avg(eps_fc_clean, firm_names, 'eps_fc', firm_ids_str)
eps_act_twenty_avg = get_twenty_yr_avg(eps_act_clean, firm_names, 'eps_act', firm_ids_str)
eod_act_twenty_avg = get_twenty_yr_avg(eod_act_clean, firm_names, 'eod_act', firm_ids_str)
eps_fc_terms_twenty_avg = get_twenty_yr_avg(eps_fc_terms_clean, firm_names, 'eps_fc_terms', firm_ids_str)

In [455]:
#put averages into a list
twenty_avg_list = [eps_fc_twenty_avg,
                  eps_act_twenty_avg,
                  eod_act_twenty_avg,
                  eps_fc_terms_twenty_avg]

In [456]:
#merge all Series
df_twenty_year_avgs = reduce(lambda x, y: pd.merge(x, y, on = 'firm_id', how = 'outer'), twenty_avg_list)

### Test

In [457]:
df_twenty_year_avgs.sample(5)

,firm_id,eps_fc,eps_act,eod_act,eps_fc_terms
261,261,1.155627,0.517884,27.285554,1.281152
394,394,0.295431,0.352524,65.600000,0.350393
242,242,0.342000,0.368156,53.994630,0.398239
488,488,0.233048,0.113171,24.628083,0.288237
151,151,1.142917,0.974371,60.768111,1.205722


In [458]:
#check there are 505 firms
df_twenty_year_avgs.firm_id.nunique()

505

In [459]:
#check missing data
df_twenty_year_avgs.isna().any()

firm_id         False
eps_fc          False
eps_act          True
eod_act         False
eps_fc_terms     True
dtype: bool

In [460]:
#check duplicate data
df_twenty_year_avgs.duplicated().any()

False

### Code
> **ISSUE 10:** No recorded yearly averages for each dataset.

**Define:**
- Create 4 separate DataFrames for all attributes
- Rename columns to "Feature_Year" (e.g. eps_fc_1999, eod_act_2000, etc.)
- Outer merge all DataFrames to create new DataFrame, `df_yearly_avgs`, on **firms** attribute

In [461]:
#helper function to calculate yearly average of each DataFrame
def calculate_df_yearly_avgs(df, col, firm_ids_str):
    #extract year from dates, reset index, and remove index name
    df = df.groupby(df[col].dt.year).mean().transpose().rename_axis('firm_id').reset_index().rename_axis(None, axis = 1)
    
    df['firm_id'] = df['firm_id'].map(firm_ids_str)
    
    return df

In [462]:
#helper function to melt columns
def melt_avgs(df, firm_id, var_name):
    return df.melt(id_vars = [firm_id],
                value_vars = df.columns.difference([firm_id]),
                var_name = var_name,
                value_name = 'average')

In [463]:
#calculate and melt
def create_df_yearly_avgs(df, col, firm_ids_str, firm_id):
    df = calculate_df_yearly_avgs(df, 'term', firm_ids_str)
    df = melt_avgs(df, firm_id, 'year')
    return df

In [469]:
#generate DataFrames of all yearly averages
df_eps_fc_yearly_avgs = create_df_yearly_avgs(eps_fc_clean, 'term', firm_ids_str, 'firm_id')
df_eps_act_yearly_avgs = create_df_yearly_avgs(eps_act_clean, 'term', firm_ids_str, 'firm_id')

#for eod_act, doesn't matter if we use date or term field
df_eod_act_yearly_avgs = create_df_yearly_avgs(eod_act_clean, 'date', firm_ids_str, 'firm_id')
df_eps_fc_terms_yearly_avgs = create_df_yearly_avgs(eps_fc_terms_clean, 'term', firm_ids_str, 'firm_id')

In [470]:
def store_df(path, df, csv_name):
    df.to_csv(path + csv_name, encoding = 'utf-8', index = False)

In [473]:
#create copies
eps_fc_yearly_avgs = df_eps_fc_yearly_avgs.copy()
eps_act_yearly_avgs = df_eps_act_yearly_avgs.copy()
eod_act_yearly_avgs = df_eod_act_yearly_avgs.copy()
eps_fc_terms_yearly_avgs = df_eps_fc_terms_yearly_avgs.copy()

In [474]:
#add new column entailing "type" of feature before concat
eps_fc_yearly_avgs = eps_fc_yearly_avgs.assign(feature = 'eps_fc')
eps_act_yearly_avgs = eps_act_yearly_avgs.assign(feature = 'eps_act')
eod_act_yearly_avgs = eod_act_yearly_avgs.assign(feature = 'eod_act')
eps_fc_terms_yearly_avgs = eps_fc_terms_yearly_avgs.assign(feature = 'eps_fc_terms')

In [475]:
yearly_avgs_arr = [eps_fc_yearly_avgs, 
                  eps_act_yearly_avgs,
                  eod_act_yearly_avgs,
                  eps_fc_terms_yearly_avgs]

In [476]:
#merge all yearly average DataFrames
df_yearly_avgs = pd.concat(yearly_avgs_arr)

### Test

In [477]:
#each average, individually

#eps_fc_yearly
eps_fc_yearly_avgs.sample(10)

,firm_id,year,average,feature
6497,437,2011,0.96625,eps_fc
3771,236,2006,0.26250,eps_fc
8118,38,2015,0.47000,eps_fc
7946,371,2014,1.60400,eps_fc
2873,348,2004,0.09425,eps_fc
8808,223,2016,-1.34600,eps_fc
5848,293,2010,0.31775,eps_fc
2905,380,2004,0.46650,eps_fc
4821,276,2008,0.23050,eps_fc
1244,234,2001,NaN,eps_fc


In [478]:
#eps_fc_yearly
eps_act_yearly_avgs.sample(10)

,firm_id,year,average,feature
2589,64,2004,0.462500,eps_act
6987,422,2012,0.905000,eps_act
5934,379,2010,0.807500,eps_act
2196,176,2003,NaN,eps_act
4312,272,2007,0.420000,eps_act
10339,239,2019,5.433333,eps_act
8269,189,2015,0.510000,eps_act
7834,259,2014,0.567500,eps_act
3905,370,2006,0.537500,eps_act
10456,356,2019,0.333333,eps_act


In [479]:
#eod_act_yearly
eod_act_yearly_avgs.sample(10)

,firm_id,year,average,feature
587,82,2000,19.296900,eod_act
7708,133,2014,70.464350,eod_act
8909,324,2016,57.950000,eod_act
9412,322,2017,24.815000,eod_act
10372,272,2019,19.286667,eod_act
10007,412,2018,257.380000,eod_act
7710,135,2014,62.512500,eod_act
4551,6,2008,16.265000,eod_act
8754,169,2016,27.937500,eod_act
2136,116,2003,NaN,eod_act


In [480]:
#eps_fc_terms_yearly
eps_fc_terms_yearly_avgs.sample(10)

,firm_id,year,average,feature
4817,272,2009,0.20575,eps_fc_terms
5492,442,2010,0.52950,eps_fc_terms
2624,99,2005,0.54000,eps_fc_terms
7111,41,2014,0.83350,eps_fc_terms
1544,29,2003,NaN,eps_fc_terms
1253,243,2002,0.20175,eps_fc_terms
1309,299,2002,NaN,eps_fc_terms
414,414,2000,0.17525,eps_fc_terms
3224,194,2006,NaN,eps_fc_terms
8556,476,2016,1.00300,eps_fc_terms


In [481]:
#verify all columns are consistent
print(eps_fc_yearly_avgs.columns, '\n',
      eps_act_yearly_avgs.columns, '\n',
      eod_act_yearly_avgs.columns, '\n',
      eps_fc_terms_yearly_avgs.columns)

Index(['firm_id', 'year', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'year', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'year', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'year', 'average', 'feature'], dtype='object')


In [482]:
#verify there are 505 firms in each dataframe
print(eps_fc_yearly_avgs.firm_id.nunique(), '\n',
      eps_act_yearly_avgs.firm_id.nunique(), '\n',
      eod_act_yearly_avgs.firm_id.nunique(), '\n',
      eps_fc_terms_yearly_avgs.firm_id.nunique())

505 
 505 
 505 
 505


In [483]:
#all yearly averages
df_yearly_avgs.sample(10)

,firm_id,year,average,feature
4883,338,2009,1.01775,eps_fc_terms
7902,327,2014,0.48800,eps_fc
6904,339,2012,0.91250,eps_act
4053,13,2008,1.08500,eps_fc_terms
2409,389,2004,NaN,eps_fc_terms
9106,16,2017,0.97000,eps_act
1360,350,2002,0.13950,eps_fc_terms
2495,475,2003,14.42170,eod_act
7214,144,2013,0.53000,eps_act
6437,377,2012,1.55000,eps_fc_terms


In [484]:
#verify 505 firms in all yearly averages df
df_yearly_avgs.firm_id.nunique()

505

### Code

> **ISSUE 11:** No recorded quarterly averages for each dataset.

**Define:** 

- Parse `eod_act_clean` dates  by **calendar quarter average** with quarterly data in a new DataFrame.
- Create separate DataFrames containing quarterly averages for `eps_fc_clean`, `eps_act_clean`, and `eps_fc_terms_clean` 
- Rename columns to "Quarter_Year_Feature" (e.g. **q1_eps_fc,** etc.)
- Outer merge all DataFrames into a new df `df_quarterly_avgs` on **firms**

In [485]:
#helper function to calculate quarterly average of each DataFrame
def calculate_df_quarterly_avgs(df, col, firm_ids_str):
    #extract quarter from Period objects
    df =  df.groupby(df[col].dt.strftime('Q%q')).mean().transpose().rename_axis('firm_id').reset_index().rename_axis(None, axis = 1)
    df['firm_id'] = df['firm_id'].map(firm_ids_str)
    
    return df

In [486]:
#calculate and melt
def create_df_quarterly_avgs(df, col, firm_ids_str, firm_id):
    df = calculate_df_quarterly_avgs(df, 'term', firm_ids_str)
    df = melt_avgs(df, firm_id, 'quarter')
    
    #lowercase quarter column
    df.quarter = df.quarter.str.lower()
    return df

In [487]:
#generate all quarterly average DataFrames
df_eps_fc_quarter_avgs = create_df_quarterly_avgs(eps_fc_clean, 'term', firm_ids_str, 'firm_id')
df_eps_act_quarter_avgs = create_df_quarterly_avgs(eps_act_clean, 'term', firm_ids_str, 'firm_id')
df_eod_act_quarter_avgs = create_df_quarterly_avgs(eod_act_clean, 'term', firm_ids_str, 'firm_id')
df_eps_fc_terms_quarter_avgs = create_df_quarterly_avgs(eps_fc_terms_clean, 'term', firm_ids_str, 'firm_id')

In [488]:
#create copies
eps_fc_quarter_avgs = df_eps_fc_quarter_avgs.copy()
eps_act_quarter_avgs = df_eps_act_quarter_avgs.copy()
eod_act_quarter_avgs = df_eod_act_quarter_avgs.copy()
eps_fc_terms_quarter_avgs = df_eps_fc_terms_quarter_avgs.copy()

In [489]:
#add new column entailing "type" of feature before concat
eps_fc_quarter_avgs = eps_fc_quarter_avgs.assign(feature = 'eps_fc')
eps_act_quarter_avgs = eps_act_quarter_avgs.assign(feature = 'eps_act')
eod_act_quarter_avgs = eod_act_quarter_avgs.assign(feature = 'eod_act')
eps_fc_terms_quarter_avgs = eps_fc_terms_quarter_avgs.assign(feature = 'eps_fc_terms')

In [490]:
#put DataFrames in array
quarterly_avgs_arr = [eps_fc_quarter_avgs,
                  eps_act_quarter_avgs,
                  eod_act_quarter_avgs,
                  eps_fc_terms_quarter_avgs]

In [491]:
#merge all quarterly average DataFrames
df_quarter_avgs = pd.concat(quarterly_avgs_arr)

### Test

In [492]:
#each quarter individually

#eps_fc_quarter
eps_fc_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
953,448,q2,0.235846,eps_fc
168,168,q1,0.659615,eps_fc
1059,49,q3,0.641476,eps_fc
235,235,q1,0.206000,eps_fc
1281,271,q3,0.707600,eps_fc
189,189,q1,0.436905,eps_fc
1694,179,q4,0.861250,eps_fc
1952,437,q4,1.121143,eps_fc
1689,174,q4,0.642095,eps_fc
1563,48,q4,1.140222,eps_fc


In [493]:
#eps_act_quarter
eps_act_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
694,189,q2,0.529206,eps_act
569,64,q2,0.884286,eps_act
1335,325,q3,0.581667,eps_act
1070,60,q3,0.453095,eps_act
122,122,q1,0.305159,eps_act
1712,197,q4,0.588000,eps_act
1286,276,q3,0.310952,eps_act
1028,18,q3,0.552738,eps_act
1270,260,q3,0.526667,eps_act
1844,329,q4,0.187167,eps_act


In [494]:
#eod_act_quarter
eod_act_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
433,433,q1,48.890524,eod_act
899,394,q2,66.740000,eod_act
764,259,q2,24.032381,eod_act
1132,122,q3,57.020000,eod_act
554,49,q2,75.435357,eod_act
1626,111,q4,63.020500,eod_act
1586,71,q4,199.566900,eod_act
994,489,q2,61.975952,eod_act
1400,390,q3,47.084448,eod_act
164,164,q1,48.588010,eod_act


In [495]:
#eps_fc_terms_quarter
eps_fc_terms_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
572,67,q2,1.985000,eps_fc_terms
1507,497,q3,1.218450,eps_fc_terms
1174,164,q3,0.456400,eps_fc_terms
1160,150,q3,1.078900,eps_fc_terms
508,3,q2,0.912950,eps_fc_terms
210,210,q1,0.844222,eps_fc_terms
1886,371,q4,1.398700,eps_fc_terms
1667,152,q4,0.595150,eps_fc_terms
378,378,q1,0.548526,eps_fc_terms
1464,454,q3,2.409071,eps_fc_terms


In [496]:
#verify that all columns are consistent
print(eps_fc_quarter_avgs.columns, '\n',
      eps_act_quarter_avgs.columns, '\n',
      eod_act_quarter_avgs.columns, '\n',
      eps_fc_terms_quarter_avgs.columns)

Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object')


In [497]:
#verify there are 505 firms in each dataframe
print(eps_fc_quarter_avgs.firm_id.nunique(), '\n',
      eps_act_quarter_avgs.firm_id.nunique(), '\n',
      eod_act_quarter_avgs.firm_id.nunique(), '\n',
      eps_fc_terms_quarter_avgs.firm_id.nunique())

505 
 505 
 505 
 505


In [498]:
#all quarterly averages
df_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
242,242,q1,0.418000,eps_fc_terms
20,20,q1,5.275800,eps_fc
1082,72,q3,0.332619,eps_fc
787,282,q2,0.328050,eps_fc_terms
1915,400,q4,0.331845,eps_act
1470,460,q3,0.918350,eps_fc_terms
1064,54,q3,2.250700,eps_fc
1159,149,q3,0.839000,eps_fc_terms
600,95,q2,0.638333,eps_act
342,342,q1,26.324367,eod_act


In [499]:
#verify 505 firms in df_quarter_avgs
df_quarter_avgs.firm_id.nunique()

505

In [500]:
df_twenty_year_avgs.head(2)


,firm_id,eps_fc,eps_act,eod_act,eps_fc_terms
0,0,0.531678,0.352169,32.896718,0.567579
1,1,0.339879,-0.388520,40.303125,0.709745


### Code
>**ISSUE 12:** Unnormalized data among `df_twenty_year_avgs`, `df_yearly_avgs`, and `df_quarter_avgs`

**Define:** 
- Normalize features across all 3 DataFrames. 
- For `df_twenty_year_avgs`, melt **feature column names** (eps_fc, eps_act, eod_act, eps_fc_terms) into a single column called **feature**

Desired output for each DataFrame:


| firm_id     | average  | average_type | time_period   | feature|
|:------------|:-------------:|:-----------:|:---------:|-------:|
|43           | 3.14          | quarterly   | q1        | eps_fc |

> ***Apply the above table format to all three DFs, then concatenate to create `all_avgs` .***

**Note:** For `df_twenty_year_avgs`, it would be redundant to put *twenty_year* as the value under both **average_type** and **time_period** columns.

To curb this, I will assign all **time_period** values as NaN for `df_twenty_year_avgs`.

In [501]:
#assign column order
avgs_col_order = ['firm_id', 'average', 'average_type', 'time_period', 'feature']

In [502]:
#create clean copies of dfs
twenty_year_clean = df_twenty_year_avgs.copy()
yearly_clean = df_yearly_avgs.copy()
quarterly_clean = df_quarter_avgs.copy()

- ### `twenty_year_clean`

In [503]:
#isolate feature names
feats_col_names = twenty_year_clean.columns.difference(['firm_id'])

In [504]:
#move eps_fc . . . eps_fc_terms under "average_type"
twenty_year_clean = pd.melt(twenty_year_clean,
        id_vars = ['firm_id'],
        value_vars = feats_col_names,
        var_name = 'feature',
        value_name = 'average')

In [505]:
#add 'feature' and 'time_period' columns
twenty_year_clean = twenty_year_clean.assign(average_type = 'twenty_year', time_period = np.nan)

In [506]:
#reorder columns
twenty_year_clean = twenty_year_clean[avgs_col_order]

### Test

In [507]:
#twenty_year
twenty_year_clean.sample(5)

,firm_id,average,average_type,time_period,feature
895,390,0.425149,twenty_year,NaN,eps_act
686,181,0.690857,twenty_year,NaN,eps_act
504,504,58.752963,twenty_year,NaN,eod_act
577,72,0.217348,twenty_year,NaN,eps_act
701,196,0.908108,twenty_year,NaN,eps_act


In [508]:
#verify 505 firm names
twenty_year_clean.firm_id.nunique()

505

In [509]:
#verify all time_period values are NaN
twenty_year_clean.time_period.unique()

array([nan])

In [510]:
#verify average_type contains only twenty_year
twenty_year_clean.average_type.unique()

array(['twenty_year'], dtype=object)

- ### `yearly_clean`

In [511]:
#assign 'average_type' column
yearly_clean = yearly_clean.assign(average_type = 'yearly')

In [512]:
#rename 'year' column to 'time_period'
yearly_clean.rename(columns = {'year' : 'time_period'}, inplace = True)

In [513]:
#reorder columns
yearly_clean = yearly_clean[avgs_col_order]

### Test

In [514]:
yearly_clean.sample(10)

,firm_id,average,average_type,time_period,feature
6222,162,1.25350,yearly,2012,eps_fc_terms
2718,193,0.17425,yearly,2004,eps_fc
3467,437,0.66675,yearly,2006,eps_fc_terms
3418,388,0.80550,yearly,2006,eps_fc_terms
8047,472,0.69850,yearly,2015,eps_fc_terms
3981,446,13.41125,yearly,2006,eod_act
10264,164,1.15575,yearly,2019,eps_fc
6340,280,0.75750,yearly,2011,eps_act
1727,212,0.03675,yearly,2002,eps_fc
4479,439,7.01625,yearly,2007,eod_act


In [515]:
#verify 505 firms
yearly_clean.firm_id.nunique()

505

In [516]:
#verify 'average_type' contains only 'yearly'
yearly_clean.average_type.unique()

array(['yearly'], dtype=object)

In [517]:
#verify 'feature' contains all features
all(feat in feats_col_names for feat in yearly_clean.feature.unique())

True

In [518]:
#verify there are 21 years recorded (1999 - 2019)
len(yearly_clean.time_period.unique())

21

In [519]:
#examine lowest and highest year
yearly_clean.time_period.min(), yearly_clean.time_period.max()

(1999, 2019)

- ### `quarterly_clean`

In [520]:
#assign 'average_type' column
quarterly_clean = quarterly_clean.assign(average_type = 'quarterly')

In [521]:
#rename 'quarter' column to 'time_period'
quarterly_clean.rename(columns = {'quarter' : 'time_period'}, inplace = True)

In [522]:
#reorder columns
quarterly_clean = quarterly_clean[avgs_col_order]

### Test

In [523]:
quarterly_clean.sample(10)

,firm_id,average,average_type,time_period,feature
640,135,0.996154,quarterly,q2,eps_fc
1116,106,1.954857,quarterly,q3,eps_fc
1454,444,0.757429,quarterly,q3,eps_fc
913,408,0.191350,quarterly,q2,eps_fc_terms
1375,365,0.721667,quarterly,q3,eps_act
581,76,18.854648,quarterly,q2,eod_act
1121,111,63.512500,quarterly,q3,eod_act
871,366,120.000000,quarterly,q2,eod_act
1407,397,33.538452,quarterly,q3,eod_act
123,123,0.600143,quarterly,q1,eps_fc


In [524]:
#verify 505 firms
quarterly_clean.firm_id.nunique()

505

In [525]:
#verify 'average_type' contains only 'quarterly'
quarterly_clean.average_type.unique()

array(['quarterly'], dtype=object)

In [526]:
#verify 'time_period' contains only 'q1' till 'q4'
quarterly_clean.time_period.unique()

array(['q1', 'q2', 'q3', 'q4'], dtype=object)

In [527]:
#verify 'feature' contains all features
all(feat in feats_col_names for feat in quarterly_clean.feature.unique())

True

### Code
>**ISSUE 13:** - 20-year, yearly, and quarterly averages contained in different DataFrames.

**Define:** Concatenate the normalized `twenty_year_clean`, `yearly_clean`, and `quarterly_clean` into new CSV **all_avgs.csv**.

In [528]:
#put all average DFs in an array
avgs_array = [twenty_year_clean, yearly_clean, quarterly_clean]

In [529]:
all_avgs = pd.concat(avgs_array)

### Test

In [530]:
all_avgs.sample(10)

,firm_id,average,average_type,time_period,feature
9873,278,0.493250,yearly,2019,eps_fc_terms
772,267,0.127500,yearly,2001,eps_fc_terms
5390,340,0.093250,yearly,2010,eps_fc_terms
6693,128,0.973250,yearly,2013,eps_fc_terms
4764,219,0.084375,yearly,2008,eps_act
9670,75,NaN,yearly,2018,eps_act
1745,230,1.061619,quarterly,q4,eps_fc
1878,363,0.166885,twenty_year,NaN,eps_fc_terms
5643,88,0.700000,yearly,2011,eps_fc_terms
4036,501,0.349250,yearly,2006,eps_fc


In [531]:
all_avgs.shape

(52015, 5)

In [532]:
#verify 505 firms, no duplicates
all_avgs.firm_id.nunique()

505

In [533]:
#verify only 3 average_types
all_avgs.average_type.unique()

array(['twenty_year', 'yearly', 'quarterly'], dtype=object)

In [534]:
#verify 4 features
all_avgs.feature.unique()

array(['eod_act', 'eps_act', 'eps_fc', 'eps_fc_terms'], dtype=object)

### Code
>**ISSUE 14:** Unnormalized data among `eps_act`, `eps_fc`, `eod_act`, and `eps_fc_terms`

**Define:**

- Achieve desired `features.csv` output:

| firm_id  | term_forecast     | value      | feature         |    date    |
|:---------| :-------: | :-----------------:| :-------------: |----------: |
| 44       |     2004Q1  |  3.55           |        eps_fc      |  NaN      |
| 55       | 2004Q4     |  6.22       |  eps_fc_terms       | 2004-07-01 |
| 202      | 2015Q1      | 6.28      |   eod_act         |   1999-03-31    |


> **date** feature contains the **date column in `eod_act`** and **forecast_made in `eps_fc_terms`**. They are assigned to the same column because both columns are consistent: the same DateTime Object.

> **date** feature will be NaN for `eps_fc` and `eps_act`.

- ### `eps_fc` and `eps_act`

> For both DataFrames, we don't have to worry about handling the **date** column. That will be resolved later when merging all the DataFrames with `eod_act` and `eps_fc_terms`

In [535]:
fc_act_order = ['firm_id', 'feature', 'term', 'value']

In [536]:
#melt firm names
eps_fc_clean = eps_fc_clean.melt (id_vars = ['term'],
                  var_name = 'firm_id')

In [537]:
eps_act_clean = eps_act_clean.melt(id_vars = ['term'],
                                  var_name = 'firm_id')

In [538]:
#reassign all firms to firm_id
eps_fc_clean['firm_id'] = eps_fc_clean['firm_id'].map(firm_names_ids)
eps_act_clean['firm_id'] = eps_act_clean['firm_id'].map(firm_names_ids)

In [539]:
#add feature column
eps_fc_clean = eps_fc_clean.assign(feature = 'eps_fc')
eps_act_clean = eps_act_clean.assign(feature = 'eps_act')

In [540]:
#reorder columns
eps_fc_clean = eps_fc_clean[fc_act_order]
eps_act_clean = eps_act_clean[fc_act_order]

### Test

In [541]:
eps_fc_clean.sample(10)

,firm_id,feature,term,value
25631,305,eps_fc,2001Q4,0.251
20630,245,eps_fc,2011Q3,0.222
23370,278,eps_fc,2003Q3,0.172
33943,404,eps_fc,2000Q4,0.067
22867,272,eps_fc,2003Q4,NaN
32115,382,eps_fc,2005Q4,0.180
36423,433,eps_fc,2011Q4,0.703
20677,246,eps_fc,2002Q2,-0.272
28548,339,eps_fc,2017Q1,0.215
25009,297,eps_fc,2014Q2,0.894


In [542]:
#verify 505 firms in both
eps_fc_clean.firm_id.nunique() == 505 & eps_act_clean.firm_id.nunique() == 505

True

In [543]:
#examine 'feature' in both
len(eps_fc_clean.feature.unique()) == 1 & len(eps_act_clean.feature.unique()) == 1

True

In [544]:
#verify 84 unique term values: 21 years x 4 quarters
len(eps_fc_clean.term.unique()), len(eps_act_clean.term.unique())

(84, 84)

- ### `eod_act`

In [545]:
#define column order
feats_col_order = ['firm_id', 'feature', 'date', 'term', 'value']

In [546]:
#melt firm ids
eod_act_clean = eod_act_clean.melt(id_vars = ['date', 'term'],
                  var_name = 'firm_id')

In [547]:
#reassign firm_ids
eod_act_clean['firm_id'] = eod_act_clean['firm_id'].map(firm_names_ids)

In [548]:
#add feature column
eod_act_clean = eod_act_clean.assign(feature = 'eod_act')

In [549]:
#reorder columns 
eod_act_clean = eod_act_clean[feats_col_order]

### Test

In [550]:
eod_act_clean.sample(5)

,firm_id,feature,date,term,value
39181,466,eod_act,2008-06-30,2008Q2,45.9153
36558,435,eod_act,2003-09-30,2003Q3,NaN
10255,122,eod_act,2000-12-29,2000Q4,NaN
20650,245,eod_act,2016-09-30,2016Q3,181.6600
971,11,eod_act,2010-12-31,2010Q4,30.0800


In [551]:
#verify 505 firms 
eod_act_clean.firm_id.nunique() == 505 

True

In [552]:
#examine 'feature' column
eod_act_clean.feature.unique()

array(['eod_act'], dtype=object)

In [553]:
#verify 84 unique dates and terms
len(eod_act_clean.term.unique()) == 84, len(eod_act_clean.date.unique()) == 84

(True, True)

- ### `eps_fc_terms`

In [554]:
#melt firm ids
eps_fc_terms_clean = eps_fc_terms_clean.melt(id_vars = ['forecast_made', 'term'],
                       var_name = 'firm_id')

In [555]:
#reassign firm ids
eps_fc_terms_clean['firm_id'] = eps_fc_terms_clean['firm_id'].map(firm_names_ids)

In [556]:
#rename forecast_made to date
eps_fc_terms_clean = eps_fc_terms_clean.rename(columns = {'forecast_made' : 'date'})

In [557]:
#add feature column
eps_fc_terms_clean = eps_fc_terms_clean.assign(feature = 'eps_fc_terms')

In [558]:
#reorder columns
eps_fc_terms_clean = eps_fc_terms_clean[feats_col_order]

### Test

In [559]:
eps_fc_terms_clean.sample(10)

,firm_id,feature,date,term,value
37647,470,eps_fc_terms,2011-07-01,2011Q4,-0.259
8949,111,eps_fc_terms,2017-01-01,2017Q2,2.008
37288,466,eps_fc_terms,2001-10-01,2002Q1,0.083
6628,82,eps_fc_terms,2016-10-01,2017Q1,0.740
21801,272,eps_fc_terms,2010-01-01,2010Q2,0.070
31578,394,eps_fc_terms,2014-04-01,2014Q3,0.163
32964,412,eps_fc_terms,2000-09-01,2001Q1,0.668
26113,326,eps_fc_terms,2008-01-01,2008Q2,1.646
20345,254,eps_fc_terms,2006-01-01,2006Q2,0.900
14965,187,eps_fc_terms,2001-01-01,2001Q2,NaN


In [560]:
#verify 505 firms
eps_fc_terms_clean.firm_id.nunique()

505

In [561]:
#examine 'feature' column
eps_fc_terms_clean.feature.unique()

array(['eps_fc_terms'], dtype=object)

In [562]:
#verify 80 unique terms (1999 not recorded)
eps_fc_terms_clean.term.nunique() == 80

True

- ### `features.csv`

Now's the time to bring everything together!

In [563]:
feat_array = [eps_fc_clean, eps_act_clean, eod_act_clean, eps_fc_terms_clean]

In [564]:
#concat to prepare for outer left merge
eod_act_eps_terms = pd.concat([eod_act_clean, eps_fc_terms_clean], join = 'outer')

In [565]:
#outer join
features = pd.concat([eod_act_eps_terms, eps_fc_clean], join = 'outer', sort = False)
features = pd.concat([features, eps_act_clean], join = 'outer', sort = False)

In [566]:
features.sample(10)

,firm_id,feature,date,term,value
39342,468,eps_act,NaT,2006Q3,0.510
11238,133,eps_act,NaT,2015Q3,NaN
22651,283,eps_fc_terms,2002-07-01,2002Q4,NaN
13579,161,eps_fc,NaT,2012Q4,0.159
34621,412,eps_fc,NaT,2002Q2,0.326
31485,374,eod_act,2016-06-30,2016Q2,52.420
10997,137,eps_fc_terms,2009-01-01,2009Q2,0.920
31131,370,eod_act,2011-12-30,2011Q4,19.510
25951,308,eod_act,2018-12-31,2018Q4,110.470
14475,172,eod_act,2005-12-30,2005Q4,21.500


### Test

In [567]:
features.sample(5)

,firm_id,feature,date,term,value
10344,123,eps_fc,NaT,2002Q1,0.332000
2996,35,eps_fc,NaT,2013Q1,2.027000
17271,205,eps_act,NaT,2011Q4,0.806667
6066,72,eps_fc,NaT,2003Q3,0.162000
17518,208,eps_act,NaT,2010Q3,NaN


In [568]:
#verify 505 terms
features.firm_id.nunique() == 505

True

In [569]:
#examine 'feature' column
features.feature.unique()

array(['eod_act', 'eps_fc_terms', 'eps_fc', 'eps_act'], dtype=object)

In [570]:
#verify 84 term ranges
features.term.nunique() == 84, 

(True,)

In [571]:
#verify 164 date values: 84 ones from eod_act, and 80 from eps_fc_terms
features.date.nunique()

164

# III) Store Data <a id = "store"></a>

**Disclaimer:** I decided to put the following CSVs under the ***./data/clean/averages/components*** directory because they form *part* of the main CSVs under the ***./data/clean/averages*** directory:
- `quarter-avgs-eod-act.csv`
- `quarter-avgs-eps-act.csv`
- `quarter-avgs-eps-fc.csv`
- `quarter-avgs-eps-fc-terms.csv`

> The above are all components of the `quarter-avgs.csv` under **/averages/**

- `yearly-avgs-eod-act.csv`
- `yearly-avgs-eps-act.csv`
- `yearly-avgs-eps-fc.csv`
- `yearly-avgs-eps-fc-terms.csv`

> The above are all components of the `yearly-avgs.csv` under **/averages/**

**Additionally, I decided to keep `avgs.csv` under the *./data/clean/averages/* directory.** Although this CSV containing all averages would be highly inefficient for the data exploration stage, I decided it would be best to keep it on hand for convenience.

Using `avgs.csv` would mean having the parse through the column names to isolate 20-year, yearly, and quarterly averages, but all of that redundance can be avoided by simply focusing on the `twenty-year-avgs`, `yearly-avgs`, and `quarter-avgs` CSVs alone.

## Firms

In [576]:
firm_ids.to_csv(PATH_CLEAN + 'firms.csv', encoding = 'utf-8', index = False)

## Averages

In [577]:
#twenty-year-averages
df_twenty_year_avgs.to_csv(PATH_CLEAN_AVGS + 'twenty-year-avgs.csv', encoding = 'utf-8', index = False)

In [578]:
#yearly averagas
df_yearly_avgs.to_csv(PATH_CLEAN_AVGS + 'yearly-avgs.csv', encoding = 'utf-8', index = False)

In [579]:
#quarterly averages
df_quarter_avgs.to_csv(PATH_CLEAN_AVGS + 'quarter-avgs.csv', encoding = 'utf-8', index = False)

In [580]:
#all averages
all_avgs.to_csv(PATH_CLEAN_AVGS + 'avgs.csv', encoding = 'utf-8', index = False)

**NOTE:** For the twenty year, yearly, and quarterly datasets, I stored their old `df_` version instead of their new clean versions. 
> This is because their old versions are stored more efficiently as ***standalone*** CSVs, and were only modified to fit the larger `all_avgs` DataFrame.

## Average Components

In [581]:
#quarterly averages
store_df(PATH_CLEAN_AVGS_SUB, df_eps_fc_quarter_avgs, 'quarter-avgs-eps-fc.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eps_act_quarter_avgs, 'quarter-avgs-eps-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eod_act_quarter_avgs, 'quarter-avgs-eod-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eps_fc_terms_quarter_avgs, 'quarter-avgs-eps-fc-terms.csv')

In [582]:
#store yearly averages
store_df(PATH_CLEAN_AVGS_SUB, df_eps_fc_yearly_avgs, 'yearly-avgs-eps-fc.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eps_act_yearly_avgs, 'yearly-avgs-eps-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eod_act_yearly_avgs, 'yearly-avgs-eod-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, df_eps_fc_terms_yearly_avgs, 'yearly-avgs-eps-fc-terms.csv')

## Features

In [ ]:
eps_fc_clean.to_csv(PATH_CLEAN + 'eps-fc.csv')
eps_act_clean.to_csv(PATH_CLEAN + 'eps-act.csv')
eod_act_clean.to_csv(PATH_CLEAN + 'eod-act.csv')
eps_fc_terms_clean.to_csv(PATH_CLEAN + 'eps-fc-terms.csv')

features.to_csv(PATH_CLEAN + 'features.csv')

# IV) Notes



- When dealing with the CSVs depicting averages, we need to keep in mind **missing data** in our interpretations.
- I decided to put the years as columns under yearly average DF generation because: the amount of columns generated here, TK, would be less than the amount of 
	firms there are, 505. [TK link to a study showing how having too many columns take up memory vs rows]
    
- keep in mind that `eod_act` is based on **calendar years** not fiscal years.

In [136]:

#convert notebook to HTML